In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from PIL import Image
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
import random

In [9]:
my_data=pd.read_csv('../data/image_data.csv')
data_array=my_data.to_numpy()
images_path='../data/pocus_images/convex/'
images_data=[]
print('processing data...')
for i in data_array:
    img = cv2.imread(images_path+i[0]+'.'+i[2], cv2.IMREAD_GRAYSCALE)
    img=cv2.equalizeHist(img)
    img=cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    img_1d=img.reshape(224*224)
    images_data.append(img_1d)


In [60]:
r=[]
conf_matricies=[]
f1_reports=[]
for i in range(10):
    r.append(random.randint(0,100))
print(r)

[66, 48, 62, 56, 71, 52, 86, 22, 65, 49]


In [61]:
number_of_models=21
X=np.asarray(images_data)
y=np.asarray(my_data['Label'])
Xandy=[]
models=[]
#creating models
for i in range(number_of_models):
    models.append(make_pipeline(StandardScaler(), SVC(gamma='auto',random_state=r[0])))

#Creating array test splits for models
#X_train, X_test, y_train, y_test 
for i in range(number_of_models):
    x = train_test_split(X, y, test_size=0.2, random_state=r[0])
    Xandy.append(x)

In [62]:
#Equalizing Data
print('Equalizing Data')
for i in range(number_of_models):
    x=Xandy[i][0]
    y=Xandy[i][2]
    cov=np.count_nonzero(y == 'covid')
    pneu=np.count_nonzero(y == 'pneumonia')
    reg=np.count_nonzero(y == 'regular')
    #deleting random pneumonia elements from each model set
    while(cov!=pneu):
        index=random.randint(0, len(x)-1)
        if(y[index]=='pneumonia'):
            x=np.delete(x,index,0)
            y=np.delete(y,index,0)
            pneu-=1
    #deleting random regular elements from each model set       
    while(cov!=reg):
        index=random.randint(0, len(x)-1)
        if(y[index]=='regular'):
            x=np.delete(x,index,0)
            y=np.delete(y,index,0)
            reg-=1
    Xandy[i][0]=x
    Xandy[i][2]=y

Equalizing Data


In [63]:
#Train each Model
print('Training Models...')
for i in range(number_of_models):
    models[i].fit(Xandy[i][0],Xandy[i][2])

Training Models...


In [64]:
#Get predicted value from xtest of each model 
predicted=[]
for i in range(number_of_models):
    predicted.append(models[i].predict(Xandy[i][1]))
predicted[0]

array(['regular', 'pneumonia', 'pneumonia', 'covid', 'pneumonia',
       'regular', 'covid', 'covid', 'covid', 'regular', 'regular',
       'regular', 'pneumonia', 'regular', 'regular', 'covid', 'pneumonia',
       'covid', 'regular', 'regular', 'regular', 'covid', 'pneumonia',
       'regular', 'pneumonia', 'regular', 'covid', 'pneumonia', 'regular',
       'regular', 'pneumonia'], dtype=object)

In [65]:
#Method to get most predicted element
from mostCommon import most_common

In [66]:
#Create 1 predict array by model voting
voted_prediction=[]
for i in range(len(predicted[0])):
    voting=[]
    for j in range(len(predicted)):
        voting.append(predicted[j][i])
    voted_prediction.append(most_common(voting))

In [67]:
conf_matrix=confusion_matrix(Xandy[0][3],voted_prediction)
conf_matricies.append(conf_matrix)

In [68]:
classification_report=metrics.classification_report(Xandy[0][3],voted_prediction, digits=3)
f1_reports.append(classification_report)

In [69]:
print(conf_matricies[0])

[[ 6  2  0]
 [ 2  8  0]
 [ 0  0 13]]


In [70]:
print(f1_reports[0])

              precision    recall  f1-score   support

       covid      0.750     0.750     0.750         8
   pneumonia      0.800     0.800     0.800        10
     regular      1.000     1.000     1.000        13

    accuracy                          0.871        31
   macro avg      0.850     0.850     0.850        31
weighted avg      0.871     0.871     0.871        31

